In [409]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [410]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [411]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(training_set)

# obtengo los nodos del grafo, esta vez los nodos son los usuarios y no los problemas:
nodes = training_set.user_id.unique()

# creo un diccionario que va a tener a los usuarios como keys y los valores seran los
# problemas que ha hecho ese usuario
grouped = training_set.groupby('user_id')['problem_id'].apply(list)

# muestra el numero de usuarios
print(len(nodes))

# muestra la lista de problemas que ha hecho cada usuario
print(grouped)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [412]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [413]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [414]:
# In[3]:

def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de usuarios
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de problemas que ha hecho cada usuario
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado


# In[4]:

def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    


# In[5]:

def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo


In [415]:
# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

16805


In [416]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    

In [417]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped.index, 'list_problem_id':grouped.values})

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)

      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [418]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))

      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [419]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users_recommend = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users_recommend)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [420]:

# en df_new tengo los usuarios a los que tengo que hacer recomendaciones

# primero guardo la lista de usuarios
user_list_recomend = df_users_recommend.user_id.unique()

# creo el nuevo dataframe con los resultados 
column_user_recomend = {'user_id': user_list_to_recommend}
dataframe_user_recomend = pd.DataFrame.from_dict(column_user_recomend)

print(dataframe_user_recomend)

    user_id
0        18
1        25
2        60
3       130
4       414
5       443
6       448
7       689
8       912
9       935
10     1711
11     1893
12     1952
13     1955
14     2025
15     2027
16     2038
17     2041
18     2051
19     2096
20     2106
21     2120
22     2257
23     2355
24     2373
25     2576
26     2626
27     2906
28     2966
29     3118
..      ...
35     3708
36     3710
37     3712
38     3714
39     3722
40     3724
41     3727
42     3731
43     3735
44     3749
45     3759
46     3779
47     3784
48     3795
49     3842
50     3846
51     3847
52     3855
53     3859
54     3873
55     3880
56     3890
57     3931
58     3937
59     3938
60     3941
61     3946
62     3961
63     3963
64     3969

[65 rows x 1 columns]


In [421]:
def getNeighbors(row, graph):
    """
        Funcion que devuelve la lista de los vecinos de ese usuario
    """
    
    # ordeno los problemas que se pueden recomendar
    column_result = graph.neighbors(row['user_id'])
    
    # ahora filtro la columna para que los usuarios esten en la lista final a utilizar
    # column_result = column_result[column_result['two'].isin(user_list_recomend) == True]
    # print(column_result)
    
    if column_result == []:
        column_result = graph.nodes()
    
    # hago el filtro de los k mejores
    return column_result

In [422]:
dataframe_user_recomend['neighbors'] = dataframe_user_recomend.apply (lambda row: getNeighbors(row, graph), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_user_recomend)

    user_id                                          neighbors
0        18                                        [1711, 103]
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...
11     1893  [322, 517, 3783, 130, 206, 847, 84, 3481, 89, ...
12     1952  [3840, 2050, 2051, 517, 518, 8, 1922, 3854, 37...
13     1955  [130, 1931, 1485, 206, 847, 1959, 62, 1949, 19...
14     2025  [2048, 3840, 2050, 2051, 517, 518, 3847, 2

In [423]:
print(linksFiltered)

[(9, 8, 8), (9, 35, 9), (9, 67, 5), (9, 84, 7), (9, 89, 9), (9, 133, 5), (9, 129, 5), (9, 191, 6), (9, 247, 6), (9, 322, 5), (9, 47, 5), (9, 206, 10), (9, 62, 11), (9, 414, 7), (9, 437, 9), (9, 130, 10), (9, 103, 11), (9, 511, 5), (9, 575, 5), (9, 610, 5), (9, 517, 5), (9, 935, 9), (9, 1190, 7), (9, 1264, 6), (9, 847, 9), (9, 1711, 11), (9, 1809, 5), (9, 1922, 8), (9, 2050, 7), (9, 1949, 8), (9, 2226, 7), (9, 2545, 5), (8, 35, 15), (8, 42, 13), (8, 46, 5), (8, 60, 5), (8, 67, 7), (8, 84, 16), (8, 89, 34), (8, 124, 9), (8, 133, 9), (8, 129, 7), (8, 152, 5), (8, 137, 8), (8, 191, 16), (8, 44, 6), (8, 39, 5), (8, 247, 12), (8, 322, 15), (8, 47, 8), (8, 325, 5), (8, 206, 43), (8, 62, 47), (8, 249, 5), (8, 409, 7), (8, 414, 18), (8, 25, 6), (8, 437, 44), (8, 443, 9), (8, 448, 7), (8, 130, 23), (8, 483, 11), (8, 103, 46), (8, 488, 10), (8, 511, 6), (8, 542, 5), (8, 562, 7), (8, 560, 8), (8, 431, 10), (8, 581, 5), (8, 584, 6), (8, 582, 6), (8, 597, 6), (8, 588, 6), (8, 589, 5), (8, 593, 5), (

In [424]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los pesos de sus enlaces
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    user = row['user_id']
    suma = 0
    
    for elem in neighbors_list:
        #print(suma)
        if graph.get_edge_data(user,elem,default=0) != 0:
            suma = suma + graph[user][elem]['weight']
        
    return suma

In [425]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada usuario vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con user_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    user = row['user_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        if graph.get_edge_data(user,elem,default=0) != 0:
            # obtengo el peso del enlace
            peso_enlace = graph[user][elem]['weight']
        else:
            peso_enlace = 0
        
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [426]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_user_recomend['total_weight'] = dataframe_user_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_user_recomend)


    user_id                                          neighbors  total_weight
0        18                                        [1711, 103]            10
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...           602
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...           506
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...          4430
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...          3388
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...          2882
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...           590
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...          1647
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...           124
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...          5090
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...          6641
11     1893  [322, 517, 3783, 130, 206, 847, 84, 3481, 89, ...           170

In [427]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_user_recomend['score'] = dataframe_user_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

# de esta forma en dataframe_user_recomend voy a tener las ponderaciones para cada vecino 
print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18                                        [1711, 103]            10   
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...           602   
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...           506   
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...          4430   
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...          3388   
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...          2882   
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...           590   
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...          1647   
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...           124   
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...          5090   
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...          6641   
11     1893  [322, 517, 3783, 130, 206, 

In [428]:
def getProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
   
    # lista resultante de la concatenacion de las listas de problemas de los usuarios similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_problems_df = df_users[df_users['user_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_problems
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [429]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
dataframe_user_recomend['list_problems'] = dataframe_user_recomend.apply (lambda row: getProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18                                        [1711, 103]            10   
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...           602   
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...           506   
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...          4430   
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...          3388   
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...          2882   
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...           590   
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...          1647   
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...           124   
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...          5090   
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...          6641   
11     1893  [322, 517, 3783, 130, 206, 

In [430]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_problems'] if not (x in conjunto_vacio or function_add(x))]

In [431]:
# ahora tengo la lista de posibles problemas a recomendar para cada usuario
# tengo que contar el numero de apariciones de cada problema en la lista de problemas

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de problemas sin repeticiones
dataframe_user_recomend['lista_problemas_unique'] = dataframe_user_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18                                        [1711, 103]            10   
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...           602   
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...           506   
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...          4430   
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...          3388   
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...          2882   
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...           590   
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...          1647   
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...           124   
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...          5090   
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...          6641   
11     1893  [322, 517, 3783, 130, 206, 

In [432]:
# elimino la columna de la lista de problemas (la que tiene repeticiones), ya que no interesa
del dataframe_user_recomend['list_problems']

print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18                                        [1711, 103]            10   
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...           602   
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...           506   
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...          4430   
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...          3388   
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...          2882   
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...           590   
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...          1647   
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...           124   
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...          5090   
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...          6641   
11     1893  [322, 517, 3783, 130, 206, 

In [433]:
# elimino la columna de total_weight, ya que no interesa
del dataframe_user_recomend['total_weight']

print(dataframe_user_recomend)

    user_id                                          neighbors  \
0        18                                        [1711, 103]   
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...   
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...   
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...   
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...   
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...   
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...   
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...   
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...   
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...   
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...   
11     1893  [322, 517, 3783, 130, 206, 847, 84, 3481, 89, ...   
12     1952  [3840, 2050, 2051, 517, 518, 8, 1922, 3854, 37...   
13     1955  [130, 1931, 1485, 206, 847, 1959, 62, 1949, 19...   
14     202

In [434]:
def getListProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_problems_df = df_users[df_users['user_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [435]:
# ahora, para la lista de problemas que se pueden recomendar, tengo que hacer la suma de las ponderaciones
# voy a crear una columna en la cual se guarde una lista de listas de problemas, en las que cada posicion coincidira con el 
# usuario que las haya hecho, sin que se guarden los problemas que ha hecho el usuario al que quiero recomendar
dataframe_user_recomend['lista_problemas_por_user'] = dataframe_user_recomend.apply(lambda row: getListProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(dataframe_user_recomend)


    user_id                                          neighbors  \
0        18                                        [1711, 103]   
1        25  [130, 517, 8, 137, 1931, 1037, 2726, 3102, 143...   
2        60  [129, 130, 1667, 133, 518, 8, 1931, 1922, 3495...   
3       130  [2048, 1026, 2051, 2053, 1031, 8, 9, 11, 12, 1...   
4       414  [2048, 1026, 2051, 2053, 8, 9, 3927, 12, 1037,...   
5       443  [2506, 1026, 515, 517, 518, 1031, 8, 3927, 103...   
6       448  [130, 515, 517, 8, 855, 2050, 912, 67, 2187, 2...   
7       689  [2050, 515, 517, 518, 7, 8, 12, 2573, 535, 25,...   
8       912  [448, 1190, 103, 8, 935, 206, 847, 1264, 2226,...   
9       935  [1026, 2051, 2053, 1031, 8, 9, 12, 1037, 1539,...   
10     1711  [2048, 1026, 2051, 1028, 2053, 2054, 7, 8, 9, ...   
11     1893  [322, 517, 3783, 130, 206, 847, 84, 3481, 89, ...   
12     1952  [3840, 2050, 2051, 517, 518, 8, 1922, 3854, 37...   
13     1955  [130, 1931, 1485, 206, 847, 1959, 62, 1949, 19...   
14     202

In [436]:

# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
df_separation = dataframe_user_recomend.groupby(['user_id']).lista_problemas_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['user_id', 'recommendation']

print(df_separation)

      user_id  recommendation
0          18              33
1          18              83
2          18             217
3          18             312
4          18             256
5          18              49
6          18               4
7          18             503
8          18             254
9          18               2
10         18             393
11         18             510
12         18             502
13         18             213
14         18              10
15         18              51
16         18             505
17         18              23
18         18              73
19         18              90
20         18               6
21         18               8
22         18              13
23         18             511
24         18              17
25         18             508
26         18              15
27         18             509
28         18             159
29         18             487
...       ...             ...
9700     3969             443
9701     3

In [437]:
def getScoring(row, df_users_recommend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de problemas que han hecho los vecinos del usuario
    list_problems_users_df = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_per_user = list(list_problems_users_df['lista_problemas_por_user'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_user = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el problema del que quiero calcular el scoring
    problem = row['recommendation']
    # print(problem)
    
    suma = 0
    k = len(list_problems_per_user)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_problems_per_user[i]
        # print(lista_a_comprobar)
        if problem in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + list_scores_per_user[i]
    
    return suma

In [438]:
# ahora voy a calcular el scoring para cada problema
df_separation['score'] = df_separation.apply(lambda row: getScoring(row, dataframe_user_recomend), axis=1)

print(df_separation)

      user_id  recommendation     score
0          18              33  1.000000
1          18              83  1.000000
2          18             217  1.000000
3          18             312  1.000000
4          18             256  1.000000
5          18              49  1.000000
6          18               4  1.000000
7          18             503  1.000000
8          18             254  1.000000
9          18               2  1.000000
10         18             393  1.000000
11         18             510  1.000000
12         18             502  1.000000
13         18             213  1.000000
14         18              10  1.000000
15         18              51  1.000000
16         18             505  1.000000
17         18              23  1.000000
18         18              73  1.000000
19         18              90  1.000000
20         18               6  1.000000
21         18               8  1.000000
22         18              13  1.000000
23         18             511  1.000000


In [439]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['user_id', 'score'], ascending=False)
print(df_separation)

      user_id  recommendation     score
9588     3969             162  0.636816
9619     3969              33  0.592040
9638     3969             327  0.592040
9578     3969             159  0.587065
9581     3969             187  0.542289
9582     3969             209  0.542289
9597     3969             136  0.537313
9587     3969             203  0.517413
9590     3969             141  0.492537
9604     3969             139  0.487562
9621     3969              44  0.487562
9576     3969              51  0.482587
9605     3969             109  0.482587
9577     3969             237  0.472637
9583     3969              70  0.472637
9606     3969             155  0.462687
9602     3969              13  0.437811
9598     3969             145  0.412935
9609     3969               2  0.407960
9579     3969             178  0.402985
9613     3969             310  0.393035
9629     3969             390  0.383085
9591     3969             171  0.383085
9669     3969             254  0.378109


In [440]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

print(df_recommend)

                                            recommendation  \
user_id                                                      
18       [33, 83, 217, 312, 256, 49, 4, 503, 254, 2, 39...   
25       [109, 2, 39, 254, 327, 183, 441, 33, 159, 209,...   
60       [109, 39, 33, 183, 187, 141, 162, 44, 209, 258...   
130      [49, 254, 258, 256, 253, 259, 307, 510, 316, 5...   
414      [39, 159, 253, 256, 390, 441, 70, 235, 471, 30...   
443      [33, 49, 187, 134, 162, 136, 155, 203, 256, 43...   
448      [109, 187, 2, 141, 39, 254, 183, 155, 258, 209...   
689      [49, 187, 150, 258, 327, 254, 203, 325, 62, 13...   
912      [109, 141, 187, 441, 258, 503, 254, 155, 159, ...   
935      [39, 44, 183, 150, 13, 251, 237, 4, 336, 437, ...   
1711     [259, 309, 251, 437, 308, 53, 352, 442, 314, 2...   
1893     [109, 162, 33, 44, 183, 136, 254, 141, 203, 13...   
1952     [150, 254, 39, 109, 187, 141, 155, 209, 49, 25...   
1955     [309, 510, 465, 183, 141, 258, 438, 235, 39, 3...   
2025    

In [441]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation'][:k])
        return value
    else:
        return row['recommendation'][:k]

In [442]:
k = 10
# ahora saco los k mejores problemas para cada usuario
df_recommend['k_recommendation'] = df_recommend.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend)

                                            recommendation  \
user_id                                                      
18       [33, 83, 217, 312, 256, 49, 4, 503, 254, 2, 39...   
25       [109, 2, 39, 254, 327, 183, 441, 33, 159, 209,...   
60       [109, 39, 33, 183, 187, 141, 162, 44, 209, 258...   
130      [49, 254, 258, 256, 253, 259, 307, 510, 316, 5...   
414      [39, 159, 253, 256, 390, 441, 70, 235, 471, 30...   
443      [33, 49, 187, 134, 162, 136, 155, 203, 256, 43...   
448      [109, 187, 2, 141, 39, 254, 183, 155, 258, 209...   
689      [49, 187, 150, 258, 327, 254, 203, 325, 62, 13...   
912      [109, 141, 187, 441, 258, 503, 254, 155, 159, ...   
935      [39, 44, 183, 150, 13, 251, 237, 4, 336, 437, ...   
1711     [259, 309, 251, 437, 308, 53, 352, 442, 314, 2...   
1893     [109, 162, 33, 44, 183, 136, 254, 141, 203, 13...   
1952     [150, 254, 39, 109, 187, 141, 155, 209, 49, 25...   
1955     [309, 510, 465, 183, 141, 258, 438, 235, 39, 3...   
2025    

In [443]:
# elimino las columnas que no me interesan
del df_recommend['recommendation']
del df_recommend['score']

print(df_recommend)

                                          k_recommendation
user_id                                                   
18             [33, 83, 217, 312, 256, 49, 4, 503, 254, 2]
25          [109, 2, 39, 254, 327, 183, 441, 33, 159, 209]
60         [109, 39, 33, 183, 187, 141, 162, 44, 209, 258]
130      [49, 254, 258, 256, 253, 259, 307, 510, 316, 508]
414       [39, 159, 253, 256, 390, 441, 70, 235, 471, 309]
443       [33, 49, 187, 134, 162, 136, 155, 203, 256, 438]
448        [109, 187, 2, 141, 39, 254, 183, 155, 258, 209]
689       [49, 187, 150, 258, 327, 254, 203, 325, 62, 139]
912      [109, 141, 187, 441, 258, 503, 254, 155, 159, ...
935          [39, 44, 183, 150, 13, 251, 237, 4, 336, 437]
1711     [259, 309, 251, 437, 308, 53, 352, 442, 314, 231]
1893      [109, 162, 33, 44, 183, 136, 254, 141, 203, 139]
1952      [150, 254, 39, 109, 187, 141, 155, 209, 49, 258]
1955      [309, 510, 465, 183, 141, 258, 438, 235, 39, 33]
2025      [209, 187, 136, 39, 109, 33, 141, 253, 134, 15

In [444]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [445]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()
list_recom_problems = df_recommend['k_recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [446]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [447]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [448]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [449]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [450]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [451]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [452]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.8769230769230769
Precision ----------
0.2861538461538462
Mrr  ----------
0.6016666666666667
Recall  ----------
0.2967661809709876
F1  ----------
0.27583018988626873


In [453]:

f = open("C:/hlocal/TFM/vot_pond", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()